In [0]:
from pyspark.sql.functions import to_date, col, expr
from pyspark.sql.types import IntegerType

# Parse OrderDate correctly
cleaned_df = retail_df.withColumn(
    "OrderDate",
    to_date(col("OrderDate"), "dd/MM/yyyy")
)

# Cast Sales and Quantity to integer if needed
cleaned_df = cleaned_df \
    .withColumn("Sales", col("Sales").cast(IntegerType())) \
    .withColumn("Quantity", col("Quantity").cast(IntegerType()))

# Drop rows with null critical columns after date parsing
cleaned_df = cleaned_df.na.drop(subset=["OrderID", "OrderDate", "Sales", "Quantity", "CustomerID"])

# Fill missing Quantity with 0 if any
cleaned_df = cleaned_df.fillna({"Quantity": 0})

# Remove duplicates
cleaned_df = cleaned_df.dropDuplicates()

# Add TotalRevenue column
cleaned_df = cleaned_df.withColumn("TotalRevenue", expr("Sales * Quantity"))

# Preview cleaned data
cleaned_df.show()
cleaned_df.printSchema()

temp_path = "abfss://processed@retailanalyticslake.dfs.core.windows.net/retail_sales_cleaned_csv_temp/"
cleaned_df.coalesce(1).write.format("csv") \
    .mode("overwrite") \
    .option("header", True) \
    .save(temp_path)

# 4. Rename the part file to retail_sales_cleaned.csv
files = dbutils.fs.ls(temp_path)
csv_part_file = [f.path for f in files if f.name.startswith("part-") and f.name.endswith(".csv")][0]
final_csv_path = "abfss://processed@retailanalyticslake.dfs.core.windows.net/retail_sales_cleaned.csv"

# Overwrite existing final file if exists
dbutils.fs.rm(final_csv_path, True)
dbutils.fs.cp(csv_part_file, final_csv_path)

print(f"✅ Cleaned CSV saved as: {final_csv_path}")


+-------+----------+------------+-----------+-----+--------+----------+------+------------+
|OrderID| OrderDate|     Product|   Category|Sales|Quantity|CustomerID|Region|TotalRevenue|
+-------+----------+------------+-----------+-----+--------+----------+------+------------+
|   1008|2024-01-16|Sony Headset|Electronics|  199|       2|   CUST008|   VIC|         398|
|   1003|2024-01-12|Office Chair|  Furniture|  199|       2|   CUST003|   QLD|         398|
|   1001|2024-01-10|   iPhone 14|Electronics| 1299|       1|   CUST001|   NSW|        1299|
|   1005|2024-01-13|Dining Table|  Furniture|  599|       1|   CUST005|   VIC|         599|
|   1004|2024-01-12| MacBook Pro|Electronics| 2499|       1|   CUST004|   NSW|        2499|
|   1002|2024-01-11|  Samsung TV|Electronics|  899|       1|   CUST002|   VIC|         899|
|   1006|2024-01-14| Dell Laptop|Electronics| 1099|       1|   CUST006|   QLD|        1099|
|   1007|2024-01-15|   Bookshelf|  Furniture|  149|       1|   CUST007|    SA|  